Start by listing some of the variables used to compute charges

In [1]:
#defining the possible values of categorical variables
stand_types = ["Wide Contact", "Narrow Contact", "Wide Remote", "Narrow Remote", 
               "Wide Satellite", "Narrow Satellite", "Light Aircraft Parking (LAP)", 
               "Long Term Remote"]
stand_categories = ["Contact", "Remote", "Satellite"]
seasons = ["summer", "winter"]

#defining the names and type of variables that appear
# guessing the possible_values...
stand_category = {"name": "Stand Category", "dtype": "string", "possible_values": stand_categories}
stand_type = {"name": "Stand Type", "dtype": "string", "possible_values": stand_types}
aircraft_parking_duration = {"name": "Aircraft Parking Duration", "dtype": "float"}
tonnes_mtow = {"name": "Tonnes MTOW", "dtype": "float"}
time_used_in_minutes = {"name": "Time in Minutes", "dtype": "float"}
is_night_time = {"name": "Is Night Time", "dtype": "boolean"}
season = {"name": "Season", "dtype": "string", "possible_values": seasons}
pre_notified_rate = {"name": "Pre-Notified Rate", "dtype": "float", "possible_values":[0, 100], "description": "This is a percentage"}
num_passengers = {"name": "Number of Passengers", "dtype": "int", "possible_values":[0, 10000], "description": "This is the number of passengers for fast track charge"}
qc_value = {"name": "QC Value", "dtype": "float", "possible_values":[0, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16], "description": "Noise Classification"}
tonnes = {"name": "Tonnes", "dtype": "float", "description": "This is the weight of the aircraft in tonnes"} #TODO: Is this different from mtow?
num_engines = {"name": "Number of Engines", "dtype": "int", "possible_values":[1, 2, 3, 4, 5, 6], "description": "This is the number of engines on the aircraft"}
nox_kg_per_engine = {"name": "NOx kg per Engine", "dtype": "float", "description": "This is the NOx emissions in kg per engine"}


Ideas for some of the formulas:

In [2]:
# the various charges that can be calculated.
# each charge corresponds to a formula that uses the above variables.
# we need to extract the numbers below. In order of difficulty:
# 1. just replace the numbers. 2. change the thresholds. 3. change the formula.

def aircraft_parking_charges(stand_type):
    pass

def runway_landing_and_take_off_charges():
    pass

def airbridge_charge(time_used_in_minutes, is_night_time=False):
    """
    Calculate the airbridge charge based on time used in minutes.
    The charge is 9.30 Euros/15 mins
    Section 3.3 of 2025
    No charge at night time (is_night_time=True).
    """
    return 9.30 * (time_used_in_minutes / 15) if not is_night_time else 0

def passenger_charge(season, stand_category):

    if season == "summer":
        if stand_category == "Contact":
            return 12.90
        elif stand_category == "Remote":
            return 6.25
        elif stand_category == "Satellite":
            return 11.70
    elif season == "winter":
        if stand_category == "Contact":
            return 9.20
        elif stand_category == "Remote":
            return 3.30
        elif stand_category == "Satellite":
            return 8
        
def transfer_passenger_charge(season):
    """
    Calculate the transfer passenger charge based on the season.
    Section 3.5 of 2025
    """
    if season == "summer":
        return 3.90
    elif season == "winter":
        return 2.80
    
def prm_charge(pre_notified_rate):
    """Calculate the PRM charge based on the pre-notified rate.
    Section 3.6 of 2025
    """
    if pre_notified_rate < 70:
        return 0.90
    elif pre_notified_rate < 80:
        return 0.85
    else:
        return 0.74
    
def fast_track_passenger_charge(num_passengers):
    """
    Calculate the fast track passenger charge based on the number of passengers.
    Section 3.7 of 2025
    #TODO: Is this per passenger?
    """
    return 0.9 * num_passengers

def noise_charges(is_night, qc_value, tonnes):
    """
    Calculate the noise charge based on whether it is night time and the QC value.
    Section 3.8 of 2025
    """
    if is_night:
        if qc_value == 0:
            return 0*tonnes if is_night else 0*tonnes
        elif qc_value == 0.125:
            return 0*tonnes if is_night else 0*tonnes
        elif qc_value == 0.25:
            return 0*tonnes if is_night else 0*tonnes
        elif qc_value == 0.25:
            return 0.0*tonnes if is_night else 0.0*tonnes
        elif qc_value == 0.5:
            return 2.0*tonnes if is_night else 0.0*tonnes
        elif qc_value == 1:
            return 4*tonnes if is_night else 1*tonnes
        elif qc_value == 2:
            return 8*tonnes if is_night else 2*tonnes
        elif qc_value == 4:
            return 12*tonnes if is_night else 4*tonnes
        elif qc_value == 8:
            return 16*tonnes if is_night else 6*tonnes
        elif qc_value == 16:
            return 20*tonnes if is_night else 8*tonnes
    else:
        return 0
    
def nox_charges(num_engines, nox_kg_per_engine):
    return num_engines * nox_kg_per_engine * 0.25

Could also try building the cost tables directly.

This might be straightforward from the markdown etc. 

This would give an intermediate stage for verification and review.

Can then compute costs from the tables...

In [13]:
import pandas as pd

# Example: copy and pasted text (markdown formatted might be better)
# Noise Charges
# QC	Set fee per Tonne 2025
# Day	Set fee per Tonne 2025
# Night
# 0	€0.00	€0.00
# 0.125	€0.00	€0.00
# 0.25	€0.00	€0.00
# 0.5	€0.00	€2.00
# 1	€1.00	€4.00
# 2	€2.00	€8.00
# 4	€4.00	€12.00
# 8	€6.00	€16.00
# 16	€8.00	€20.00

noise_charge_table = pd.DataFrame({
    "QC": [0, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16],
    "Set fee per Tonne 2025 Day": [0.00, 0.00, 0.00, 2.00, 4.00, 8.00, 12.00, 16.00, 20.00],
    "Set fee per Tonne 2025 Night": [0.00, 0.00, 0.00, 2.00, 4.00, 8.00, 12.00, 16.00, 20.00]
})


In [14]:
noise_charge_table

,QC,Set fee per Tonne 2025 Day,Set fee per Tonne 2025 Night
0,0.000,0.0,0.0
1,0.125,0.0,0.0
2,0.250,0.0,0.0
3,0.500,2.0,2.0
4,1.000,4.0,4.0
5,2.000,8.0,8.0
6,4.000,12.0,12.0
7,8.000,16.0,16.0
8,16.000,20.0,20.0


In [15]:
# PRM charge
# Charging basis	Pre- notified rate	Passenger charge
# PRM charge (€)	<70%	0.90
# <70-80%	0.85
# >80%	0.74

prm_charge_table = pd.DataFrame({
    "Pre-notified rate": ["<70%", "<70-80%", ">80%"],
    "PRM charge (€)": [0.90, 0.85, 0.74]
})


In [16]:
prm_charge_table

,Pre-notified rate,PRM charge (€)
0,<70%,0.90
1,<70-80%,0.85
2,>80%,0.74
